# ibm_db.exec_immediate()

## Purpose:

Prepare and execute an SQL statement, using values supplied for parameter markers that were coded in the statement<br>(if any).

## Syntax:

`IBM_DBStatement ibm_db.exec_immediate( IBM_DBConnection `*`connection,`*` string `*`statement`*` [, dictionary `*`options`*`] )`

## Parameters:

* __*connection*__&nbsp; &nbsp; &nbsp; &nbsp; : A valid Db2 server or database connection.
* __*SQLstatement*__&nbsp; &nbsp;: A valid, executable SQL statement (that may or may not contain one or more parameter markers).
* __options__&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; : A dictionary containing key-value pairs for the statement attributes that are to be set when the statement provided in the __*SQLstatement*__ parameter is prepared. Valid keys and values are:<p>
        
    * `ibm_db.SQL_ATTR_CURSOR_TYPE`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: Specifies the type of cursor that is to be used for processing result sets. Valid values are:
        * `ibm_db.SQL_CURSOR_FORWARD_ONLY`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: A forward only cursor should be used (i.e., a cursor that only scrolls forward).
        * `ibm_db. SQL_CURSOR_KEYSET_DRIVEN`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: A keyset-driven cursor should be used.
        * `ibm_db.SQL_CURSOR_STATIC`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: A static cursor should be used.
        * `ibm_db.SQL_CURSOR_DYNAMIC`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: A dynamic, scrollable cursor that detects all changes to the result set, should be used. Dynamic cursors are only supported by Db2 for z/OS servers.
    * `ibm_db.SQL_ATTR_ROWCOUNT_PREFETCH`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: Enables Db2 to determine the number of rows that are returned by a query (so the entire result set can be prefetched into memory, when possible); `ibm_db.SQL_ROWCOUNT_PREFETCH_ON` will turn this behavior __ON__ and `ibm_db.SQL_ROWCOUNT_PREFETCH_OFF` will turn it __OFF__.
    * `ibm_db.SQL_ATTR_QUERY_TIMEOUT`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: The number of seconds to wait for an SQL statement to execute before aborting and returning to the application. This option can be used to terminate long running queries &mdash; the default value of __0__ means an application will wait indefinitely for the server to complete execution.

## Return values:

* If __successful__, a valid `IBM_DBStatement` object (that contains a result set if the SQL statement specified in the __*SQLstatement*__ parameter was a __SELECT__ statement).
* If __unsuccessful__, the value `False`.

## Description:

The __ibm_db.exec_immediate()__ API is used to prepare and execute an SQL statement, using values supplied for parameter markers that were coded in the statement (if there are any).<p>
    
If you plan to execute the same SQL statement repeatedly (with or without parameter marker values), consider using the __ibm_db.prepare()__ and __ibm_db.execute()__ APIs instead. Refer to the notebook for __ibm_db.prepare()__ (i.e., the file *ibm_db-prepare.ipynb*) for a brief discussion of the advantages of using __ibm_db.prepare()__ and __ibm_db.execute()__, as opposed to __ibm_db.exec_immediate()__ to perform SQL operations against a Db2 database.<p>
    
If the SQL statement executed returns a result set, a single row can be retrieved and stored in a tuple or dictionary using the  __ibm_db.fetch_tuple()__ (tuple), __ibm_db.fetch_assoc()__ (dictionary), or __ibm_db.fetch_both()__ (tuple *and* dictionary) APIs. Alternately, the __ibm_db.fetch_row()__ API can be used to move the result set pointer to each row in the result set produced and the __ibm_db.result()__ API can be used to fetch a column from the current row.<p> 

It is important to note that if you interject Python variables into the SQL statement used with this API, you run the risk of creating an application that is vunerable to SQL injection attacks . Consequently, if you want to execute SQL statements that contain parameter markers, it is recommended that you use the __ibm_db.prepare()__ and __ibm_db.execute()__ APIs to avoid creating a potential security exposure.<p>

## Example:

In [1]:
#----------------------------------------------------------------------------------------------#
#  NAME:     ibm_db-exec_immediate.py                                                          #
#                                                                                              #
#  PURPOSE:  This program is designed to illustrate how to use the ibm_db.exec_immediate()     #
#            API.                                                                              #
#                                                                                              #
#            Additional APIs used:                                                             #
#                 ibm_db.fetch_row()                                                           #
#                 ibm_db.result()                                                              #
#                                                                                              #
#----------------------------------------------------------------------------------------------#
#                     DISCLAIMER OF WARRANTIES AND LIMITATION OF LIABILITY                     #
#                                                                                              #
#  (C) COPYRIGHT International Business Machines Corp. 2018, 2019 All Rights Reserved          #
#  Licensed Materials - Property of IBM                                                        #
#                                                                                              #
#  US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA    #
#  ADP Schedule Contract with IBM Corp.                                                        #
#                                                                                              #
#  The following source code ("Sample") is owned by International Business Machines            #
#  Corporation ("IBM") or one of its subsidiaries and is copyrighted and licensed, not sold.   #
#  You may use, copy, modify, and distribute the Sample in any form without payment to IBM,    #
#  for the purpose of assisting you in the creation of Python applications using the ibm_db    #
#  library.                                                                                    #
#                                                                                              #
#  The Sample code is provided to you on an "AS IS" basis, without warranty of any kind. IBM   #
#  HEREBY EXPRESSLY DISCLAIMS ALL WARRANTIES, EITHER EXPRESS OR IMPLIED, INCLUDING, BUT NOT    #
#  LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. #
#  Some jurisdictions do not allow for the exclusion or limitation of implied warranties, so   #
#  the above limitations or exclusions may not apply to you. IBM shall not be liable for any   #
#  damages you suffer as a result of using, copying, modifying or distributing the Sample,     #
#  even if IBM has been advised of the possibility of such damages.                            #
#----------------------------------------------------------------------------------------------#

# Load The Appropriate Python Modules
import sys         # Provides Information About Python Interpreter Constants And Functions
import ibm_db      # Contains The APIs Needed To Work With Db2 Databases

#----------------------------------------------------------------------------------------------#
# Import The Db2ConnectionMgr Class Definition, Attributes, And Methods That Have Been Defined #
# In The File Named "ibm_db_tools.py"; This Class Contains The Programming Logic Needed To     #
# Establish And Terminate A Connection To A Db2 Server Or Database                             #
#----------------------------------------------------------------------------------------------#
from ibm_db_tools import Db2ConnectionMgr

#----------------------------------------------------------------------------------------------#
# Import The ipynb_exit Class Definition, Attributes, And Methods That Have Been Defined In    #
# The File Named "ipynb_exit.py"; This Class Contains The Programming Logic Needed To Allow    #
# "exit()" Functionality To Work Without Raising An Error Or Stopping The Kernel If The        #
# Application Is Invoked In A Jupyter Notebook                                                 #
#----------------------------------------------------------------------------------------------#
from ipynb_exit import exit

# Define And Initialize The Appropriate Variables
dbName = "SAMPLE"
userID = "db2inst1"
passWord = "Passw0rd"
dbConnection = None
resultSet = False

# Create An Instance Of The Db2ConnectionMgr Class And Use It To Connect To A Db2 Database
conn = Db2ConnectionMgr('DB', dbName, '', '', userID, passWord)
conn.openConnection()
if conn.returnCode is True:
    dbConnection = conn.connectionID
else:
    conn.closeConnection()
    exit(-1)

# Define The SQL Statement That Is To Be Executed
sqlStatement = "SELECT projno, projname FROM project WHERE prstaff < 2"

# Execute The SQL Statement Just Defined
print("Executing the SQL statement \"" + sqlStatement + "\" ... ", end="")
try:
    resultSet = ibm_db.exec_immediate(dbConnection, sqlStatement)
except Exception:
    pass

# If The SQL Statement Could Not Be Executed, Display An Error Message And Exit 
if resultSet is False:
    print("\nERROR: Unable to execute the SQL statement specified.\n")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message
else:
    print("Done!\n")

# Display A Report Header
print("Query results:\n")
print("PROJNO  PROJNAME")
print("______  _____________________")

# As Long As There Are Records In The Result Set Produced, ...
while (ibm_db.fetch_row(resultSet) is True):

    # Extract The Data Value For Each Column From The Current Record
    projectNum = ibm_db.result(resultSet, 0)
    projectName = ibm_db.result(resultSet, 1)

    # Format And Display The Data Retrieved
    print("{:6}  {:24}" .format(projectNum, projectName))

# Add A Blank Line To The End Of The Report
print()

# Close The Database Connection That Was Opened Earlier
conn.closeConnection()

# Return Control To The Operating System
exit()


Connecting to the SAMPLE database ... Done!

Executing the SQL statement "SELECT projno, projname FROM project WHERE prstaff < 2" ... Done!

Query results:

PROJNO  PROJNAME
______  _____________________
AD3112  PERSONNEL PROGRAMMING   
IF2000  USER EDUCATION          
OP2011  SCP SYSTEMS SUPPORT     
OP2012  APPLICATIONS SUPPORT    
OP2013  DB/DC SUPPORT           
PL2100  WELD LINE PLANNING      

Disconnecting from the SAMPLE database ... Done!

